This notebook is used to visualise the processed events.

In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 8})
sys.path.append(os.path.dirname(os.path.abspath('./')))
from src_visualisation.visual_utils.utils_dynamic import *

path_processed = '../ProcessedData/SHRP2/'

In [ ]:
meta_both = pd.read_csv(path_processed + 'metadata_birdseye.csv').set_index('event_id')
meta_both = meta_both[(meta_both['ego_reconstructed'].astype(bool))&
                      (meta_both['surrounding_reconstructed'].astype(bool))]
event_categories = meta_both['event_category'].value_counts().index.values[::-1]
meta_both['event_category'].value_counts()

In [ ]:
event_type = event_categories[4]
print(event_type)
data_ego = pd.read_hdf(path_processed + event_type + '/Ego_birdseye.h5', key='data')
data_sur = pd.read_hdf(path_processed + event_type + '/Surrounding_birdseye.h5', key='data')

### Visualize BEV trajectories

In [ ]:
event_list = data_sur[data_sur['v_ekf']<0.1]['event_id'].unique()

In [ ]:
i = -1

In [ ]:
i += 1
event_id = event_list[i] #data_ego['event_id'].unique()[i]

df_ego = data_ego[data_ego['event_id'] == event_id]
df_sur = data_sur[data_sur['event_id'] == event_id]

print(df_sur['target_id'].nunique())
if len(df_sur)==0:
    print('No surrounding data collected for trip ', event_id)
else:
    visualize_trip(df_ego, df_sur, event_id, ekf=True) # ekf=True for EKF data, False for raw data